<h1> AWS Resource Monitor ChatBot</h1>  
각 셀을 실행만 시키시면 py 파일이 생성되도록 되어있습니다.  

쭉 실행시키시고 파일 생성이 확인되시면 마지막 셀에서 streamlit을 실행시키실 수 있습니다.

<h2> 필요한 라이브러리 셋업 </h2>  
추가로 더 필요한 라이브러리가 있을수 있으니 필요시에는 아래 requierements 에 추가하시고 실행하시면 됩니다.

In [ ]:
%%writefile requirements.txt
streamlit
boto3
pandas
plotly
anthropic

In [ ]:
!pip install -r requirements.txt

<h2> AWS 서비스 정보 활용 </h2>

In [ ]:
%%writefile aws_services.py
import boto3
import json
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import plotly.express as px
from concurrent.futures import ThreadPoolExecutor


# class 초기화 및 기본 설정
## AWS 서비스들과 상호작용하기 위한 boto3 client들을 초기화
## CloudWatch, EC2, RDS, Lambda, S3, Cost Exploere 서비스들에 대한 클라이언트 생성

class AWSResourceCollector:
    def __init__(self):
        print("Initializing AWSResourceCollector")
        self.cloudwatch = boto3.client('cloudwatch')
        self.ec2 = boto3.client('ec2')
        self.rds = boto3.client('rds')
        self.lambda_client = boto3.client('lambda')
        self.s3 = boto3.client('s3')
        self.ce = boto3.client('ce')
        
        self.service_mapping = {
            'EC2': 'Amazon Elastic Compute Cloud - Compute',
            'RDS': 'Amazon Relational Database Service',
            'Lambda': 'AWS Lambda',
            'S3': 'Amazon Simple Storage Service'
        }
        
        # 모든 리전 목록 가져오기
        try:
            self.regions = [region['RegionName'] for region in self.ec2.describe_regions()['Regions']]
        except Exception as e:
            print(f"Error getting regions: {str(e)}")
            self.regions = ['us-east-1', 'us-west-2', 'ap-northeast-2']  # 기본 리전
            
        print("AWSResourceCollector initialized")

# CloudWatch metric 수집
## 각 AWS 리소스의 성능 메트릭을 수집
## CPU 사용률, 네트웤 I/O, 디스크사용량 등 서비스별 주요 메트릭을 가져옴
## 최근 1시간 동안의 데이터 수집 --> Customizing 필요한 경우 꼭 바꿔주세요!
    
    def get_cloudwatch_metrics(self, resource_id, service_type, region, period=3600):
        """CloudWatch 메트릭 데이터 수집"""
        try:
            cloudwatch = boto3.client('cloudwatch', region_name=region)
            metrics_data = {}
            end_time = datetime.utcnow()
            start_time = end_time - timedelta(hours=1)

            metric_configs = {
                'EC2': {
                    'namespace': 'AWS/EC2',
                    'dimension_name': 'InstanceId',
                    'metrics': [
                        ('CPUUtilization', 'Percent'),
                        ('NetworkIn', 'Bytes'),
                        ('NetworkOut', 'Bytes'),
                        ('DiskReadBytes', 'Bytes'),
                        ('DiskWriteBytes', 'Bytes')
                    ]
                },
                'RDS': {
                    'namespace': 'AWS/RDS',
                    'dimension_name': 'DBInstanceIdentifier',
                    'metrics': [
                        ('CPUUtilization', 'Percent'),
                        ('FreeableMemory', 'Bytes'),
                        ('DatabaseConnections', 'Count'),
                        ('ReadIOPS', 'Count/Second'),
                        ('WriteIOPS', 'Count/Second')
                    ]
                },
                'Lambda': {
                    'namespace': 'AWS/Lambda',
                    'dimension_name': 'FunctionName',
                    'metrics': [
                        ('Invocations', 'Count'),
                        ('Duration', 'Milliseconds'),
                        ('Errors', 'Count'),
                        ('Throttles', 'Count')
                    ]
                }
            }

            if service_type in metric_configs:
                config = metric_configs[service_type]
                dimension = [{'Name': config['dimension_name'], 'Value': resource_id}]

                for metric_name, unit in config['metrics']:
                    try:
                        response = cloudwatch.get_metric_statistics(
                            Namespace=config['namespace'],
                            MetricName=metric_name,
                            Dimensions=dimension,
                            StartTime=start_time,
                            EndTime=end_time,
                            Period=period,
                            Statistics=['Average']
                        )

                        if response['Datapoints']:
                            metrics_data[metric_name] = {
                                'value': round(response['Datapoints'][-1]['Average'], 2),
                                'unit': unit
                            }
                    except Exception as e:
                        print(f"Error getting metric {metric_name}: {str(e)}")

            return metrics_data

        except Exception as e:
            print(f"Error getting CloudWatch metrics: {str(e)}")
            return {}


# 리소스 비용 조회
## AWS Cost Explorer를 사용하여 특정 리소스의 비용 정보 조회
## 최근 30일간의 비용 데이터를 가져옴 --> 이부분도 Customizing 시에 변경해주세요!
    
    def get_resource_cost(self, resource_id, service_type, region):
        """리소스별 비용 조회"""
        try:
            end_date = datetime.now()
            start_date = end_date - timedelta(days=30)
            
            response = self.ce.get_cost_and_usage(
                TimePeriod={
                    'Start': start_date.strftime('%Y-%m-%d'),
                    'End': end_date.strftime('%Y-%m-%d')
                },
                Granularity='MONTHLY',
                Metrics=['UnblendedCost'],
                Filter={
                    'And': [
                        {'Dimensions': {'Key': 'REGION', 'Values': [region]}},
                        {'Dimensions': {'Key': 'SERVICE', 'Values': [self.service_mapping[service_type]]}}
                    ]
                }
            )
            
            if response['ResultsByTime']:
                return float(response['ResultsByTime'][0]['Total']['UnblendedCost']['Amount'])
            return 0.0
            
        except Exception as e:
            print(f"Error getting resource cost: {str(e)}")
            return 0.0


# EC2 데이터 수집
## 모든 리전의 EC2 instance 정보를 수집
## 인스턴스 ID, Status, Type, IP주소 등의 상세 정보를 수집 --> 필요시 이부분도 원하시는대로 변경해주세요
## CloudWatch 메트릭과 비용정보도 함께 수집
    
    def collect_ec2_data(self):
        """EC2 인스턴스 데이터 수집"""
        print("Collecting EC2 data...")
        try:
            ec2_data = []
            
            for region in self.regions:
                try:
                    ec2_client = boto3.client('ec2', region_name=region)
                    response = ec2_client.describe_instances()
                    
                    for reservation in response['Reservations']:
                        for instance in reservation['Instances']:
                            try:
                                metrics = self.get_cloudwatch_metrics(
                                    instance['InstanceId'], 
                                    'EC2', 
                                    region
                                )
                                
                                cost = self.get_resource_cost(
                                    instance['InstanceId'],
                                    'EC2',
                                    region
                                )
                                
                                instance_data = {
                                    'resource_id': instance['InstanceId'],
                                    'service_type': 'EC2',
                                    'region': region,
                                    'status': instance['State']['Name'],
                                    'creation_date': instance['LaunchTime'].strftime('%Y-%m-%d %H:%M:%S'),
                                    'last_modified': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                                    'tags': json.dumps({tag['Key']: tag['Value'] for tag in instance.get('Tags', [])}),
                                    'cost': cost,
                                    'details': {
                                        'instance_type': instance['InstanceType'],
                                        'private_ip': instance.get('PrivateIpAddress', ''),
                                        'public_ip': instance.get('PublicIpAddress', ''),
                                        'vpc_id': instance.get('VpcId', ''),
                                        'subnet_id': instance.get('SubnetId', ''),
                                        'metrics': metrics
                                    }
                                }
                                ec2_data.append(instance_data)
                            except Exception as e:
                                print(f"Error processing EC2 instance {instance['InstanceId']}: {str(e)}")
                                continue
                except Exception as e:
                    print(f"Error processing region {region}: {str(e)}")
                    continue
            
            return pd.DataFrame(ec2_data)
            
        except Exception as e:
            print(f"Error collecting EC2 data: {str(e)}")
            return pd.DataFrame()


# RDS 데이터 수집
## 모든 리전의 RDS 데이터베이스 인스턴스 정보 수집
## 데이터베이스 엔진, 버전, 스토리지 크기 등의 정보 수집
    
    def collect_rds_data(self):
        """RDS 인스턴스 데이터 수집"""
        print("Collecting RDS data...")
        try:
            rds_data = []
            
            for region in self.regions:
                try:
                    rds_client = boto3.client('rds', region_name=region)
                    response = rds_client.describe_db_instances()
                    
                    for instance in response['DBInstances']:
                        try:
                            metrics = self.get_cloudwatch_metrics(
                                instance['DBInstanceIdentifier'],
                                'RDS',
                                region
                            )
                            
                            cost = self.get_resource_cost(
                                instance['DBInstanceIdentifier'],
                                'RDS',
                                region
                            )
                            
                            instance_data = {
                                'resource_id': instance['DBInstanceIdentifier'],
                                'service_type': 'RDS',
                                'region': region,
                                'status': instance['DBInstanceStatus'],
                                'creation_date': instance['InstanceCreateTime'].strftime('%Y-%m-%d %H:%M:%S'),
                                'last_modified': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                                'tags': json.dumps({tag['Key']: tag['Value'] for tag in instance.get('TagList', [])}),
                                'cost': cost,
                                'details': {
                                    'engine': instance['Engine'],
                                    'engine_version': instance['EngineVersion'],
                                    'instance_class': instance['DBInstanceClass'],
                                    'storage': instance['AllocatedStorage'],
                                    'endpoint': instance.get('Endpoint', {}).get('Address', ''),
                                    'metrics': metrics
                                }
                            }
                            rds_data.append(instance_data)
                        except Exception as e:
                            print(f"Error processing RDS instance {instance['DBInstanceIdentifier']}: {str(e)}")
                            continue
                except Exception as e:
                    print(f"Error processing region {region}: {str(e)}")
                    continue
            
            return pd.DataFrame(rds_data)
            
        except Exception as e:
            print(f"Error collecting RDS data: {str(e)}")
            return pd.DataFrame()


# Lambda 데이터 수집
## 모든 리전의 Lambda 함수 정보를 수집
## 함수명, 런타임, 메모리 설정, 타임아웃 정보 수집
    
    def collect_lambda_data(self):
        """Lambda 함수 데이터 수집"""
        print("Collecting Lambda data...")
        try:
            lambda_data = []
            
            for region in self.regions:
                try:
                    lambda_client = boto3.client('lambda', region_name=region)
                    paginator = lambda_client.get_paginator('list_functions')
                    
                    for page in paginator.paginate():
                        for function in page['Functions']:
                            try:
                                metrics = self.get_cloudwatch_metrics(
                                    function['FunctionName'],
                                    'Lambda',
                                    region
                                )
                                
                                cost = self.get_resource_cost(
                                    function['FunctionName'],
                                    'Lambda',
                                    region
                                )
                                
                                # 태그 정보 가져오기
                                tags_response = lambda_client.list_tags(
                                    Resource=function['FunctionArn']
                                )
                                
                                # LastModified 처리
                                if isinstance(function['LastModified'], str):
                                    last_modified = function['LastModified']
                                else:
                                    try:
                                        last_modified = function['LastModified'].strftime('%Y-%m-%d %H:%M:%S')
                                    except:
                                        last_modified = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                                
                                function_data = {
                                    'resource_id': function['FunctionName'],
                                    'service_type': 'Lambda',
                                    'region': region,
                                    'status': 'Active',
                                    'creation_date': last_modified,
                                    'last_modified': last_modified,
                                    'tags': json.dumps(tags_response.get('Tags', {})),
                                    'cost': cost,
                                    'details': {
                                        'runtime': function.get('Runtime', ''),
                                        'memory': function.get('MemorySize', 0),
                                        'timeout': function.get('Timeout', 0),
                                        'handler': function.get('Handler', ''),
                                        'metrics': metrics
                                    }
                                }
                                lambda_data.append(function_data)
                            except Exception as e:
                                print(f"Error processing Lambda function {function['FunctionName']}: {str(e)}")
                                continue
                except Exception as e:
                    print(f"Error processing region {region}: {str(e)}")
                    continue
            
            return pd.DataFrame(lambda_data)
            
        except Exception as e:
            print(f"Error collecting Lambda data: {str(e)}")
            return pd.DataFrame()

# S3 데이터 수집
## 모든 S3 버킷의 정보를 수집
## 버킷 이름, 생성일, 태그 등의 정보 수집
    
    def collect_s3_data(self):
        """S3 버킷 데이터 수집"""
        print("Collecting S3 data...")
        try:
            s3_data = []
            
            response = self.s3.list_buckets()
            
            for bucket in response['Buckets']:
                try:
                    # 버킷 리전 확인
                    region = self.s3.get_bucket_location(Bucket=bucket['Name'])
                    region = region['LocationConstraint'] or 'us-east-1'
                    
                    # 버킷 태그 가져오기
                    try:
                        tags_response = self.s3.get_bucket_tagging(Bucket=bucket['Name'])
                        tags = {tag['Key']: tag['Value'] for tag in tags_response['TagSet']}
                    except:
                        tags = {}
                    
                    cost = self.get_resource_cost(
                        bucket['Name'],
                        'S3',
                        region
                    )
                    
                    bucket_data = {
                        'resource_id': bucket['Name'],
                        'service_type': 'S3',
                        'region': region,
                        'status': 'Active',
                        'creation_date': bucket['CreationDate'].strftime('%Y-%m-%d %H:%M:%S'),
                        'last_modified': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'tags': json.dumps(tags),
                        'cost': cost,
                        'details': {
                            'creation_date': bucket['CreationDate'].strftime('%Y-%m-%d %H:%M:%S')
                        }
                    }
                    s3_data.append(bucket_data)
                except Exception as e:
                    print(f"Error processing bucket {bucket['Name']}: {str(e)}")
                    continue
            
            return pd.DataFrame(s3_data)
            
        except Exception as e:
            print(f"Error collecting S3 data: {str(e)}")
            return pd.DataFrame()

# 병렬로 모든 리소스 데이터 수집
    
    def collect_all_resources(self):
        """모든 리소스 데이터 수집"""
        print("Collecting all resources...")
        dfs = []
        collection_methods = [
            self.collect_ec2_data,
            self.collect_rds_data,
            self.collect_lambda_data,
            self.collect_s3_data
        ]
        
        # 병렬로 데이터 수집
        with ThreadPoolExecutor(max_workers=4) as executor:
            results = list(executor.map(lambda method: method(), collection_methods))
            
        for df in results:
            if not df.empty:
                dfs.append(df)
        
        result = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
        print(f"All resources collected: {len(result)} resources")
        return result


# 비용 분석 및 예측
## 서비스별, 리전별, 일별 비용 데이터를 수집하고 분석
## 향후 비용 예측을 위한 데이터 생성
    
    def get_cost_analysis(self):
        """비용 분석 데이터 수집"""
        print("Getting cost analysis...")
        try:
            end_date = datetime.now()
            start_date = end_date - timedelta(days=30)
            
            # 서비스별 비용
            service_response = self.ce.get_cost_and_usage(
                TimePeriod={
                    'Start': start_date.strftime('%Y-%m-%d'),
                    'End': end_date.strftime('%Y-%m-%d')
                },
                Granularity='MONTHLY',
                Metrics=['UnblendedCost'],
                GroupBy=[{'Type': 'DIMENSION', 'Key': 'SERVICE'}]
            )
            
            service_costs = pd.DataFrame([
                {
                    'SERVICE': group['Keys'][0],
                    'cost': float(group['Metrics']['UnblendedCost']['Amount'])
                }
                for group in service_response['ResultsByTime'][0]['Groups']
            ])
            
            # 리전별 비용
            region_response = self.ce.get_cost_and_usage(
                TimePeriod={
                    'Start': start_date.strftime('%Y-%m-%d'),
                    'End': end_date.strftime('%Y-%m-%d')
                },
                Granularity='MONTHLY',
                Metrics=['UnblendedCost'],
                GroupBy=[{'Type': 'DIMENSION', 'Key': 'REGION'}]
            )
            
            region_costs = pd.DataFrame([
                {
                    'REGION': group['Keys'][0],
                    'cost': float(group['Metrics']['UnblendedCost']['Amount'])
                }
                for group in region_response['ResultsByTime'][0]['Groups']
            ])
            
            # 일별 비용
            daily_response = self.ce.get_cost_and_usage(
                TimePeriod={
                    'Start': start_date.strftime('%Y-%m-%d'),
                    'End': end_date.strftime('%Y-%m-%d')
                },
                Granularity='DAILY',
                Metrics=['UnblendedCost'],
                GroupBy=[{'Type': 'DIMENSION', 'Key': 'SERVICE'}]
            )
            
            daily_costs = []
            for time_period in daily_response['ResultsByTime']:
                date = time_period['TimePeriod']['Start']
                for group in time_period['Groups']:
                    daily_costs.append({
                        'date': date,
                        'SERVICE': group['Keys'][0],
                        'cost': float(group['Metrics']['UnblendedCost']['Amount'])
                    })
            
            daily_costs_df = pd.DataFrame(daily_costs)
            
            return {
                'service_costs': service_costs,
                'region_costs': region_costs,
                'daily_costs': daily_costs_df
            }
            
        except Exception as e:
            print(f"Error in cost analysis: {str(e)}")
            return None


# 비용예측
## 현재 간단한 로직으로 구성하였지만 해커톤 이후 예측 모델을 개발하시거나 혹은 가지고 계신 모델이 있으시면 이부분에 embedding 하시면 편리하게 활용하실 수 있습니다.
## days 도 필요시 변경해주세요!
    
    def predict_costs(self, days=30):
        """비용 예측"""
        print("Predicting costs...")
        try:
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)
            
            # 서비스별 비용 데이터 수집
            response = self.ce.get_cost_and_usage(
                TimePeriod={
                    'Start': start_date.strftime('%Y-%m-%d'),
                    'End': end_date.strftime('%Y-%m-%d')
                },
                Granularity='DAILY',
                Metrics=['UnblendedCost'],
                GroupBy=[{'Type': 'DIMENSION', 'Key': 'SERVICE'}]
            )
            
            # 서비스별 비용 예측
            predictions = {}
            for service in self.service_mapping.keys():
                service_costs = []
                dates = []
                for time_period in response['ResultsByTime']:
                    for group in time_period['Groups']:
                        if group['Keys'][0] == self.service_mapping[service]:
                            cost = float(group['Metrics']['UnblendedCost']['Amount'])
                            if cost > 0:  # 0이 아닌 비용만 포함
                                service_costs.append(cost)
                                dates.append(pd.to_datetime(time_period['TimePeriod']['Start']))
                
                if service_costs:
                    df = pd.DataFrame({'date': dates, 'cost': service_costs})
                    df = df.set_index('date')
                    
                    # 간단한 선형 회귀로 변경
                    current_daily_avg = df['cost'].mean()
                    if len(df) > 1:
                        trend = (df['cost'].iloc[-1] - df['cost'].iloc[0]) / len(df)
                    else:
                        trend = 0
                    
                    predicted_daily_avg = current_daily_avg + trend
                    
                    predictions[service] = {
                        'current_daily_avg': current_daily_avg,
                        'predicted_daily_avg': max(0, predicted_daily_avg),  # 음수 방지
                        'trend': 'increasing' if trend > 0 else 'decreasing' if trend < 0 else 'stable',
                        'predicted_next_month': max(0, predicted_daily_avg * 30)
                    }
            
            print(f"Cost predictions generated: {predictions}")
            return predictions
            
        except Exception as e:
            print(f"Error predicting costs: {str(e)}")
            return None

# 활용을 위한 최적화 추천
## 리소스 사용 패턴을 활용해서 비용 최적화 추천사항을 생성 (향후 로직 embedding  )
## 저사용 인스턴스, 중지된 리소스 등을 식별
    
    def get_optimization_recommendations(self):
        """리소스 최적화 추천"""
        print("Getting optimization recommendations...")
        try:
            recommendations = []
            
            # EC2 인스턴스 분석
            ec2_data = self.collect_ec2_data()
            if not ec2_data.empty:
                for _, instance in ec2_data.iterrows():
                    metrics = instance['details'].get('metrics', {})
                    
                    # CPU 사용률 기반 추천
                    if 'CPUUtilization' in metrics:
                        cpu_util = metrics['CPUUtilization']['value']
                        if cpu_util < 20:
                            recommendations.append({
                                'resource_id': instance['resource_id'],
                                'service_type': 'EC2',
                                'recommendation_type': 'Downsizing',
                                'reason': f'Low CPU utilization ({cpu_util}%)',
                                'potential_savings': instance['cost'] * 0.5,
                                'action': 'Consider using a smaller instance type'
                            })
                    
                    # 중지된 인스턴스 확인
                    if instance['status'] == 'stopped':
                        recommendations.append({
                            'resource_id': instance['resource_id'],
                            'service_type': 'EC2',
                            'recommendation_type': 'Termination',
                            'reason': 'Instance is stopped',
                            'potential_savings': instance['cost'],
                            'action': 'Consider terminating if not needed'
                        })
            
            # RDS 인스턴스 분석
            rds_data = self.collect_rds_data()
            if not rds_data.empty:
                for _, instance in rds_data.iterrows():
                    metrics = instance['details'].get('metrics', {})
                    
                    # 연결 수 기반 추천
                    if 'DatabaseConnections' in metrics:
                        connections = metrics['DatabaseConnections']['value']
                        if connections < 5:
                            recommendations.append({
                                'resource_id': instance['resource_id'],
                                'service_type': 'RDS',
                                'recommendation_type': 'Downsizing',
                                'reason': f'Low number of connections ({connections})',
                                'potential_savings': instance['cost'] * 0.4,
                                'action': 'Consider using a smaller instance class'
                            })
            
            print(f"Recommendations generated: {recommendations}")
            return pd.DataFrame(recommendations)
            
        except Exception as e:
            print(f"Error generating recommendations: {str(e)}")
            return pd.DataFrame()

# 시각화 (비용)
## Plotly를 사용하여 비용 데이터를 시각화
## 파이차트, 라인 차트, 바 차트 등 생성 (취향에 맞게 나중에 재구성 해셔도 좋을 것 같습니다!)
    
    def create_cost_visualizations(self):
        """비용 시각화"""
        print("Creating cost visualizations...")
        try:
            cost_data = self.get_cost_analysis()
            if not cost_data:
                print("No cost data available")
                return None
                
            visualizations = {}
            
            # 서비스별 비용 파이 차트
            fig_pie = px.pie(
                cost_data['service_costs'],
                values='cost',
                names='SERVICE',
                title='Cost Distribution by Service'
            )
            visualizations['service_pie'] = fig_pie
            
            # 일별 비용 트렌드 라인 차트
            fig_line = px.line(
                cost_data['daily_costs'],
                x='date',
                y='cost',
                color='SERVICE',
                title='Daily Cost Trend by Service'
            )
            visualizations['daily_trend'] = fig_line
            
            # 리전별 비용 바 차트
            fig_bar = px.bar(
                cost_data['region_costs'],
                x='REGION',
                y='cost',
                title='Cost by Region'
            )
            visualizations['region_bar'] = fig_bar
            
            print(f"Visualizations created: {list(visualizations.keys())}")
            return visualizations
                
        except Exception as e:
            print(f"Error creating visualizations: {str(e)}")
            return None


<h2> Bedrock 활용 </h2>


In [ ]:
%%writefile bedrock_utils.py
import boto3
import json
import os
import pandas as pd


class BedrockService:
# 클래스 초기화
## AWS Bedrock 서비스 클라이언트 초기화
## us-west-2 리전 사용 (리전은 oregon으로 설정했습니다)
## Claude 3 Sonnet 을 기본 모델로 설정 (변경 필요하면 시도해보셔도 좋습니다)

    def __init__(self):
        self.bedrock_runtime = boto3.client(
            service_name='bedrock-runtime',
            region_name='us-west-2'
        )
        self.model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

# 모델 호출
## Bedrock 모델 호출
## Prompt: 입력 텍스트
## max_tokens : 최대 응답 토큰 수
## temperature : 응답의 창의성 정도 (실험 필요시 조정하여 활용하실 수 있습니다)
    
    def invoke_model(self, prompt, max_tokens=1000, temperature=0.7):
        try:
            body = {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": max_tokens,
                "messages": [
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                "temperature": temperature
            }
            
            response = self.bedrock_runtime.invoke_model(
                modelId=self.model_id,
                body=json.dumps(body)
            )
            
            response_body = json.loads(response['body'].read())
            return response_body['content'][0]['text']
            
        except Exception as e:
            print(f"Error invoking Bedrock model: {str(e)}")
            return None

# open search 연결 전 레벨에서의 자연어 쿼리 처리
## 자연어로 된 쿼리를 AWS 리소스 필터 파라미터로 전환
## 반환값: service_type, region, status를 포함하는 JSON 객체
## JSON 파싱로직 포함

    def process_natural_language_query(self, query):
        try:
            prompt = f"""
            Convert this natural language query to AWS resource filter parameters.
            Query: {query}
            
            Return only a JSON object with these exact fields:
            {{
                "service_type": "EC2" or "RDS" or "Lambda" or "S3",
                "region": "region name if specified, otherwise null",
                "status": "status if specified, otherwise null"
            }}
            """
            
            response = self.invoke_model(prompt, max_tokens=500, temperature=0)
            if response:
                try:
                    # JSON 문자열에서 실제 JSON 객체 부분만 추출
                    json_str = response.strip()
                    if '{' in json_str and '}' in json_str:
                        json_str = json_str[json_str.find('{'):json_str.rfind('}')+1]
                    return json.loads(json_str)
                except json.JSONDecodeError as e:
                    print(f"Error parsing JSON response: {str(e)}")
                    return {
                        "service_type": None,
                        "region": None,
                        "status": None
                    }
            return None
        except Exception as e:
            print(f"Error in process_natural_language_query: {str(e)}")
            return None

# 비용 인사이트 생성
## AWS 비용 데이터를 분석하여 인사이트 제공
## 주요 비용 요인, 비정상패턴, 최적화 기회, 트랜드 등 분석
## DataFrame을 dict로 변환하여 처리함

    def generate_cost_insights(self, cost_data):
        try:
            cost_data_dict = {
                'service_costs': cost_data['service_costs'].to_dict(orient='records'),
                'region_costs': cost_data['region_costs'].to_dict(orient='records'),
                'daily_costs': cost_data['daily_costs'].to_dict(orient='records') if 'daily_costs' in cost_data else []
            }
    
            prompt = f"""
            다음 AWS 비용 데이터를 분석하여 상세한 인사이트를 제공해주세요:
            {json.dumps(cost_data_dict)}
            
            다음 항목들에 대해 분석해주세요:
            1. 주요 비용 발생 요인
            2. 비정상적인 패턴이나 급격한 비용 증가
            3. 비용 최적화가 가능한 영역
            4. 전반적인 비용 추세와 향후 예측
            
            분석 결과를 다음과 같은 형식으로 제공해주세요:
    
            ### 주요 비용 발생 요인
            - [구체적인 분석 내용]
    
            ### 이상 패턴 분석
            - [비정상적인 비용 패턴 설명]
    
            ### 최적화 기회
            - [구체적인 최적화 방안]
    
            ### 비용 추세
            - [추세 분석 및 예측]
            """
            return self.invoke_model(prompt, max_tokens=1500, temperature=0.3)
        except Exception as e:
            print(f"Error generating cost insights: {str(e)}")
            return "현재 비용 분석을 생성할 수 없습니다."
    

#추천 사항 강화
## AWS 리소스에 대한 최적화 추천사항 제공
## 구체적인 행동 계획, 비용 절감 가능성, 성능 영향 등 포함
## DataFrame 혹은 dict 형태의 데이터 처리 가능
    
    def enhance_recommendations(self, resource_data):
        try:
            if isinstance(resource_data, pd.DataFrame):
                resource_data = resource_data.to_dict(orient='records')
    
            prompt = f"""
            다음 AWS 리소스에 대한 상세한 최적화 전략을 제공해주세요:
            {json.dumps(resource_data)}
            
            다음 내용을 포함하여 자연스러운 문장으로 작성해주세요:
    
            1. 현재 상황 분석과 문제점
            2. 구체적인 최적화 방안과 기대효과
            3. 예상되는 비용 절감 효과
            4. 구현 시 고려사항과 주의점
            5. AWS 모범 사례 기반의 권장사항
    
            기술적인 내용을 포함하되, 이해하기 쉽게 설명해주세요.
            단계별 나열이나 목록 형태를 피하고, 자연스러운 문단 형태로 작성해주세요.
            """
            return self.invoke_model(prompt, max_tokens=1000, temperature=0.7)
        except Exception as e:
            print(f"Error enhancing recommendations: {str(e)}")
            return "현재 추천 사항을 생성할 수 없습니다."




# AWS 전문가 채팅
## AWS 관련 질문에 대한 전문가 수준의 응답제공
## 추가 컨텍스트 정보 활용
## 기술적이면서도 이해하기 쉬운 응답 생성
    
    def chat_with_aws_expert(self, user_question, context=None):
        try:
            # context가 DataFrame인 경우 dict로 변환
            if isinstance(context, pd.DataFrame):
                context = context.to_dict(orient='records')
            elif isinstance(context, dict):
                for key, value in context.items():
                    if isinstance(value, pd.DataFrame):
                        context[key] = value.to_dict(orient='records')

            prompt = f"""
            You are an AWS expert. Answer this question about AWS resources:
            Question: {user_question}
            
            Context (if available):
            {json.dumps(context) if context else 'No additional context provided'}
            
            Provide a detailed, technical, yet easy to understand response.
            """
            return self.invoke_model(prompt, max_tokens=2000, temperature=0.7)
        except Exception as e:
            print(f"Error in chat with AWS expert: {str(e)}")
            return "Unable to process your question at this time."


<h2> Chatbot 구축 </h2>

In [ ]:
%%writefile app.py
import streamlit as st
import boto3
import json
import pandas as pd
from datetime import datetime
from aws_services import AWSResourceCollector
from bedrock_utils import BedrockService
import plotly.express as px

# 초기 설정
## Chatbot명, icon, layout 설정

st.set_page_config(
    page_title="AWS Resource Monitor",
    page_icon="☁️",
    layout="wide"
)

# 디버그 모드 설정
DEBUG = True

def debug_print(message):
    if DEBUG:
        print(f"DEBUG: {message}")

# Bedrock 서비스 초기화
bedrock_service = BedrockService()

# 캐시 데코레이터
## 성능 최적화를 위한 캐시 함수 정의
## AWS 리소스, 비용 분석, 예측, 추천 데이터를 캐시 (5분) --> customizing 필요시 바꿔주세요!

@st.cache_data(ttl=300)
def fetch_aws_resources():
    debug_print("Fetching AWS resources...")
    collector = AWSResourceCollector()
    resources = collector.collect_all_resources()
    return resources

@st.cache_data(ttl=300)
def fetch_cost_analysis():
    debug_print("Fetching cost analysis...")
    collector = AWSResourceCollector()
    analysis = collector.get_cost_analysis()
    return analysis

@st.cache_data(ttl=300)
def fetch_cost_predictions():
    debug_print("Fetching cost predictions...")
    collector = AWSResourceCollector()
    predictions = collector.predict_costs()
    return predictions

@st.cache_data(ttl=300)
def fetch_recommendations():
    debug_print("Fetching recommendations...")
    collector = AWSResourceCollector()
    recommendations = collector.get_optimization_recommendations()
    return recommendations


# DatabaseConnection 클래스
## AWS리소스 데이터 관리
## 자연어 쿼리 처리 및 필터링 로직

class DatabaseConnection:
    def __init__(self):
        debug_print("Initializing DatabaseConnection")
        self.resources_df = fetch_aws_resources()
        
    def execute_query(self, query):
        debug_print(f"Executing query: {query}")
        try:
            filtered_df = self.resources_df.copy()
            
            # Bedrock을 통한 쿼리 파라미터 추출
            query_params = bedrock_service.process_natural_language_query(query)
            debug_print(f"Query parameters: {query_params}")
            
            if query_params:
                # 서비스 타입 필터링
                if query_params.get('service_type'):
                    filtered_df = filtered_df[filtered_df['service_type'] == query_params['service_type']]
                
                # 리전 필터링
                if query_params.get('region'):
                    filtered_df = filtered_df[filtered_df['region'] == query_params['region']]
                
                # 상태 필터링
                if query_params.get('status'):
                    filtered_df = filtered_df[filtered_df['status'].str.lower() == query_params['status'].lower()]
            
            return filtered_df
            
        except Exception as e:
            debug_print(f"Error executing query: {str(e)}")
            return self.resources_df  # 오류 발생 시 전체 데이터 반환

# 제목
st.title("☁️ AWS Resource Monitor")

# 탭 생성
tab1, tab2, tab3, tab4, tab5 = st.tabs([
    "Resource Query",
    "Cost Analysis",
    "Resource Metrics",
    "Optimization",
    "AWS Expert Chat"
])

# 탭 1: Resource Query
## 리소스 쿼리 인터페이스
## 샘플 쿼리 버튼
## 결과 표시 및 상세 정보 보기

with tab1:
    debug_print("Rendering Resource Query tab")
    
    with st.sidebar:
        st.header("Sample Queries")
        sample_queries = [
            "us-west-2 리전의 모든 EC2 인스턴스 보기",
            "us-west-2 리전의 RDS 리소스 목록",
            "Lambda 함수 목록 보기",
            "모든 S3 버킷 조회",
            "실행 중인 EC2 인스턴스 보기"
        ]
        
        for query in sample_queries:
            if st.button(query):
                st.session_state['user_input'] = query
                
        st.header("Quick Filters")
        service_filter = st.multiselect(
            "Filter by Service",
            ["EC2", "RDS", "Lambda", "S3"],
            default=None
        )
    

    user_input = st.text_area(
        "Enter your question about AWS resources:",
        value=st.session_state.get('user_input', ''),
        height=100,
        placeholder="Example: Show me all EC2 instances in us-west-2"
    )

    if st.button("Query Resources", type="primary"):
        if user_input:
            debug_print(f"Processing query: {user_input}")
            try:
                db = DatabaseConnection()
                results = db.execute_query(user_input)
                
                if results is not None and not results.empty:
                    st.subheader("Query Results:")
                    
                    if service_filter:
                        results = results[results['service_type'].isin(service_filter)]
                    
                    # 기본 정보 표시
                    display_cols = ['resource_id', 'service_type', 'region', 'status']
                    if 'cost' in results.columns:
                        display_cols.append('cost')
                    
                    st.dataframe(
                        results[display_cols],
                        use_container_width=True,
                        hide_index=True
                    )
                    
                    # 리소스 상세 정보 표시
                    if not results.empty:
                        st.subheader("Resource Details")
                        selected_resource = st.selectbox(
                            "Select a resource to view details:",
                            results['resource_id'].tolist()
                        )
                        
                        if selected_resource:
                            resource_data = results[results['resource_id'] == selected_resource].iloc[0]
                            
                            col1, col2, col3 = st.columns(3)
                            with col1:
                                st.metric("Service Type", resource_data['service_type'])
                            with col2:
                                st.metric("Status", resource_data['status'])
                            with col3:
                                if 'cost' in resource_data:
                                    st.metric("Cost (30 days)", f"${resource_data['cost']:.2f}")
                            
                            # 상세 정보 표시
                            if isinstance(resource_data.get('details'), dict):
                                st.json(resource_data['details'])
                            
                            # 태그 정보 표시
                            if resource_data.get('tags'):
                                st.subheader("Tags")
                                try:
                                    tags = json.loads(resource_data['tags'])
                                    st.json(tags)
                                except:
                                    st.json(resource_data['tags'])
                    
                    st.success("Query executed successfully!")
                else:
                    st.info("No results found for this query.")
                    
            except Exception as e:
                st.error(f"Error processing query: {str(e)}")
                debug_print(f"Error details: {str(e)}")
        else:
            st.warning("Please enter a query first.")

# 탭 2: Cost Analysis
## 비용 분석 대시보드
## 예측 및 트렌드 표시
## Plotly 사용하여 시각화 수행

with tab2:
    debug_print("Rendering Cost Analysis tab")
    st.header("Cost Analysis Dashboard")
    
    predictions = fetch_cost_predictions()
    if isinstance(predictions, dict) and predictions:
        st.subheader("💰 Cost Predictions")
        cols = st.columns(len(predictions))
        for idx, (service, pred_data) in enumerate(predictions.items()):
            with cols[idx]:
                st.metric(
                    label=f"{service} Cost Trend",
                    value=f"${pred_data['current_daily_avg']:.2f}/day",
                    delta=f"${pred_data['predicted_daily_avg'] - pred_data['current_daily_avg']:.2f}"
                )
                st.caption(f"Trend: {pred_data['trend']}")
    
    cost_data = fetch_cost_analysis()
    if isinstance(cost_data, dict) and cost_data:
        st.subheader("📊 Cost Analysis")
        total_cost = cost_data['service_costs']['cost'].sum()
        st.metric("Total Cost", f"${total_cost:,.2f}")
        
        col1, col2 = st.columns(2)
        with col1:
            st.markdown("**Service Costs**")
            st.dataframe(
                cost_data['service_costs'].style.format({'cost': '${:,.2f}'}),
                use_container_width=True,
                hide_index=True
            )
        with col2:
            st.markdown("**Region Costs**")
            st.dataframe(
                cost_data['region_costs'].style.format({'cost': '${:,.2f}'}),
                use_container_width=True,
                hide_index=True
            )
        
        st.subheader("🤖 AI Cost Insights")
        insights = bedrock_service.generate_cost_insights(cost_data)
        if insights:
            st.markdown(insights)
        else:
            st.info("현재 비용 분석 데이터를 생성할 수 없습니다.")

        st.subheader("📈 Cost Visualizations")
        
        fig_pie = px.pie(
            cost_data['service_costs'],
            values='cost',
            names='SERVICE',
            title='Cost Distribution by Service'
        )
        st.plotly_chart(fig_pie, use_container_width=True)
        
        fig_bar = px.bar(
            cost_data['region_costs'],
            x='REGION',
            y='cost',
            title='Cost by Region'
        )
        st.plotly_chart(fig_bar, use_container_width=True)
        
        if 'daily_costs' in cost_data:
            fig_line = px.line(
                cost_data['daily_costs'],
                x='date',
                y='cost',
                color='SERVICE',
                title='Daily Cost Trend'
            )
            st.plotly_chart(fig_line, use_container_width=True)
    else:
        st.info("No cost analysis data available")

# 탭 3: Resource Metrics
## 서비스별 리소스 메트릭 표시
## 확장 가능한 메트릭 뷰 구성

with tab3:
    debug_print("Rendering Resource Metrics tab")
    st.header("Resource Metrics Dashboard")
    
    selected_service = st.selectbox(
        "Select Service",
        ["EC2", "RDS", "Lambda"]
    )
    
    resources_df = fetch_aws_resources()
    if not resources_df.empty:
        service_resources = resources_df[resources_df['service_type'] == selected_service]
        
        for _, resource in service_resources.iterrows():
            with st.expander(f"{resource['resource_id']} Metrics"):
                if isinstance(resource.get('details'), dict):
                    metrics = resource['details'].get('metrics', {})
                    if metrics:
                        metric_cols = st.columns(len(metrics))
                        for i, (metric_name, metric_data) in enumerate(metrics.items()):
                            with metric_cols[i]:
                                st.metric(
                                    metric_name,
                                    f"{metric_data['value']} {metric_data['unit']}"
                                )
                    else:
                        st.info("No metrics available for this resource")
                else:
                    st.info("No metrics available for this resource")

# 탭 4: Optimization
## 리소스 최적화 추천
## AI기반 상세 전략 제공

with tab4:
    debug_print("Rendering Optimization tab")
    st.header("Resource Optimization Recommendations")
    
    recommendations = fetch_recommendations()
    if not recommendations.empty:
        total_savings = recommendations['potential_savings'].sum()
        st.metric("예상 총 절감액", f"${total_savings:.2f}")
        
        for _, rec in recommendations.iterrows():
            with st.expander(f"{rec['resource_id']} ({rec['service_type']})에 대한 추천사항"):
                col1, col2 = st.columns(2)
                with col1:
                    st.markdown(f"**유형:** {rec['recommendation_type']}")
                    st.markdown(f"**사유:** {rec['reason']}")
                with col2:
                    st.markdown(f"**예상 절감액:** ${rec['potential_savings']:.2f}")
                    st.markdown(f"**권장 조치:** {rec['action']}")
                
                st.subheader("🤖 AI-Generated Optimization Strategy")
                detailed_strategy = bedrock_service.enhance_recommendations(rec.to_dict())
                if detailed_strategy:
                    st.markdown(detailed_strategy)
                else:
                    st.info("현재 최적화 전략을 생성할 수 없습니다.")
    else:
        st.info("현재 가능한 최적화 추천사항이 없습니다.")


# 탭 5: AWS Expert Chat
## AI전문가와의 채팅 인터페이스 추가
## 채팅 히스토리 관리
## 컨텍스트 기반 응답

with tab5:
    st.header("💬 Chat with AWS Expert")
    
    if 'chat_history' not in st.session_state:
        st.session_state.chat_history = []
    
    user_question = st.text_input("Ask anything about AWS:", key="aws_expert_input")
    
    if st.button("Ask Expert", key="ask_expert_button"):
        if user_question:
            try:
                # DataFrame을 dict로 변환
                resources_df = fetch_aws_resources()
                cost_analysis = fetch_cost_analysis()
                
                context = {
                    'resources': resources_df.to_dict(orient='records') if not resources_df.empty else [],
                    'cost_data': {
                        'service_costs': cost_analysis['service_costs'].to_dict(orient='records') if isinstance(cost_analysis, dict) and 'service_costs' in cost_analysis else [],
                        'region_costs': cost_analysis['region_costs'].to_dict(orient='records') if isinstance(cost_analysis, dict) and 'region_costs' in cost_analysis else [],
                        'daily_costs': cost_analysis['daily_costs'].to_dict(orient='records') if isinstance(cost_analysis, dict) and 'daily_costs' in cost_analysis else []
                    } if cost_analysis else {}
                }
                
                # 컨텍스트 데이터 로깅
                debug_print(f"Context data structure: {json.dumps(context, indent=2)}")
                
                response = bedrock_service.chat_with_aws_expert(user_question, context)
                
                if response:
                    st.session_state.chat_history.append({
                        "question": user_question,
                        "answer": response
                    })
                    
                    # 최신 응답 표시
                    st.markdown(f"**Q:** {user_question}")
                    st.markdown(f"**A:** {response}")
                    st.markdown("---")
                else:
                    st.error("Failed to get response from AWS Expert")
                    
            except Exception as e:
                st.error(f"Error processing request: {str(e)}")
                debug_print(f"Error details: {str(e)}")
        else:
            st.warning("Please enter a question first.")
    
    # 이전 채팅 히스토리 표시
    if st.session_state.chat_history:
        st.subheader("Previous Conversations")
        for chat in reversed(st.session_state.chat_history[:-1]):  # 최신 응답 제외
            st.markdown(f"**Q:** {chat['question']}")
            st.markdown(f"**A:** {chat['answer']}")
            st.markdown("---")


# 도움말
with st.expander("ℹ️ 도움말"):
    st.markdown("""
    ### 사용 방법
    1. AWS 리소스에 대한 질문을 자연어로 입력하세요
    2. '리소스 조회' 버튼을 클릭하여 결과를 확인하세요
    3. 특정 리소스에 대한 상세 정보를 확인하세요
    4. 비용 분석 및 최적화 추천 사항을 검토하세요
    
    ### 지원되는 서비스
    - EC2 (가상 서버 컴퓨팅)
    - RDS (관계형 데이터베이스)
    - Lambda (서버리스 컴퓨팅)
    - S3 (클라우드 스토리지)
    
    ### 주요 기능
    - 실시간 AWS 리소스 모니터링
    - 비용 분석 및 예측
    - 리소스 성능 지표 시각화
    - 최적화 추천 사항 제공
    - AI 기반 AWS 전문가 상담
    
    ### 데이터 업데이트
    - 리소스 정보: 5분마다 자동 갱신
    - 비용 데이터: 일일 단위 업데이트
    - 성능 지표: 실시간 수집
    
    ### 문의 사항
    - 기술 지원이 필요한 경우 AWS 전문가 채팅을 이용해주세요
    - 상세한 분석이 필요한 경우 비용 분석 탭을 확인해주세요
    """)

# 푸터
st.markdown("---")
st.markdown("이 Chatbot은 Streamlit과 Amazon Bedrock Claude 3.5 Sonnet을 기반으로 제작되었습니다")




<h2> Streamlit 실행 </h2>   
- 아래 셀을 실행 시키시면 Local URL, Network URL, External URL 등이 뜨는데 가장 뒤의 숫자 네자리(예: 8501)을 기억해두시고,<br>
- 현재 페이지 상단의 주소창에서 jupterlab/default/ 뒤에 proxy/8501/ 을 추가하고 엔터 치시면 <br>   
- streamlit 챗봇으로 넘어갑니다.   <br>
- 챗봇을 중지 시키실시에는 지금 주피터노프북 상단에서 런버튼과 새로고침 버튼 사이의 네모를 눌러주시면 중지가능합니다.   <br>
- 혹시 터미널에서 실행시키셨다면 컨트롤+c 누르시면 중지됩니다.<br>

In [ ]:
!streamlit run app.py

<h2>  필요한 IAM 권한 </h2>
bedrock:InvokeModel
기타 AWS 서비스 관련 권한들 (EC2, RDS, Lambda, S3, CloudWatch, Cost Explorer 등)

<h2> AWS Expert 탭에서는 아래와 같은 질문을 활용해보셔도 좋습니다 </h2>  
-비용 최적화 조언  

"현재 EC2 인스턴스들의 사용 패턴을 분석해서 비용 절감 방안을 제시해주세요"  

-성능 개선 가이드  
"RDS 인스턴스의 현재 메트릭을 보고 성능 개선 방안을 알려주세요"

-보안 강화 조언  
"현재 구성된 AWS 리소스들의 보안 취약점과 개선 방안을 알려주세요"

-아키텍처 리뷰  
"현재 구성된 리소스들의 아키텍처를 검토하고 개선점을 제안해주세요"

<h2> 사용환경에 따라 fetch시의 딜레이가
    있을 수 있으나 정상적으로 작동하니 기다려주시면 됩니다 </h2>  


-Resource query 첫 호출할 때

-AWS Expert 탭 넘어갈 때 혹은 답변 제시할 때  
